# Unet을 이용한 Image Segmentation 

레퍼런스 : https://colab.research.google.com/github/dhrim/MDC_2021/blob/master/material/deep_learning/unet_segmentation_multi_label.ipynb#scrollTo=cHnifLferK9Z

텐서플로우에서 데이터 받아올 때 gpu 사용하는 듯 
 수정 필요 

# 데이터 로드 

In [2]:
import tensorflow_datasets as tfds 
import numpy as np 
import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
import torch.nn as nn 
import torch.nn.functional as F 
from torchvision import transforms
import matplotlib.pyplot as plt 
#데이터 로드 

def image_data_load():
    ds =tfds.load('lost_and_found',split='train',batch_size= 200 )
    dataset = next(iter(ds))
    images = dataset['image_left'].numpy()
    labels = dataset['segmentation_label'].numpy()
    return images ,labels 

class divide(nn.Module):
    def __init__(self,divide_value):
        super().__init__()
        self.divide_value = divide_value

    def forward(self,img):
        return img/self.divide_value

def image_transform():
    transform = transforms.Compose([
        divide(255),
        transforms.ToTensor(),
        transforms.Resize((256,256)),
        transforms.Normalize(mean=0.5,std=0.5)
    ])
    return transform 

class Dset(Dataset):
    def __init__(self,images,labels,transform):
        super().__init__()
        self.images = images 
        self.labels = labels 
        self.transform = transform
    
    def __len__(self):
        return len(self.images)

    def image_transform(self,img):
        return self.transform(img)

    def __getitem__(self,idx):
        image = self.image_transform(self.images[idx])
        label = self.image_transform(self.labels[idx])

        return image, label 

def train_valid_split(images,labels):
    length = len(images)
    split_index = int(length*0.8)
    train_images,test_images = images[:split_index], images[split_index:]
    train_labels,test_labels = labels[:split_index], labels[split_index:]
    return train_images,test_images,train_labels,test_labels 

class Conv_Block(nn.Module):
    def __init__(self,input_c,output_c):
        super().__init__()
        self.conv_in = nn.Conv2d(in_channels=input_c,out_channels=output_c,kernel_size=3,padding=1)
        self.conv = nn.Conv2d(in_channels=output_c,out_channels=output_c,kernel_size=3,padding=1)
        self.batchnorm = nn.BatchNorm2d(output_c)
        self.maxpool = nn.MaxPool2d(2)

    def forward(self,x):
        x = self.conv_in(x)
        x = F.relu(x)
        x = self.batchnorm(x)
        x = self.conv(x)
        x = F.relu(x)
        conv = self.batchnorm(x)
        pool = self.maxpool(conv)
        return conv,pool

class Down(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv_Block(3,32)
        self.conv2 = Conv_Block(32,64)
        self.conv3 = Conv_Block(64,128)
        self.conv4 = Conv_Block(128,256)
        self.conv5 = self.Conv_last(256,512)

    def Conv_last(self,input_c,output_c):
        block = nn.Sequential(
                                nn.Conv2d(input_c,output_c,3,padding=1),
                                nn.ReLU(),
                                nn.BatchNorm2d(output_c),
                                nn.Conv2d(output_c,output_c,3,padding=1),
                                nn.ReLU(),
                                nn.BatchNorm2d(output_c)
        )
        return block 

    def forward(self,x):
        conv1,pool1 = self.conv1(x)
        conv2,pool2 = self.conv2(pool1)
        conv3,pool3 = self.conv3(pool2)
        conv4,pool4 = self.conv4(pool3)
        conv5 = self.conv5(pool4)
        return conv1,conv2,conv3,conv4,conv5

class Conv_up_block(nn.Module):
    def __init__(self,input_c,output_c):
        super().__init__()
        self.up_sample = nn.ConvTranspose2d(input_c,output_c,2,stride=2,padding=0)
        self.conv1 = nn.Conv2d(input_c,output_c,3,padding=1)
        self.conv2 = nn.Conv2d(output_c,output_c,3,padding=1)
        self.batchnorm = nn.BatchNorm2d(output_c)


    def forward(self,x,conv):
        self.up = self.up_sample(x)
        x = torch.cat((self.up,conv),dim=1)

        x = self.conv1(x)
        x = F.relu(x)
        x = self.batchnorm(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.batchnorm(x)
        
        return x 

class Up(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_up1 = Conv_up_block(512,256)
        self.conv_up2 = Conv_up_block(256,128)
        self.conv_up3 = Conv_up_block(128,64)
        self.conv_up4 = Conv_up_block(64,32)
        self.conv_last = nn.Conv2d(32,1,1,padding=0)
    
    def forward(self,conv1,conv2,conv3,conv4,conv5):
        up = self.conv_up1(conv5,conv4)
        up = self.conv_up2(up,conv3)
        up = self.conv_up3(up,conv2)
        up = self.conv_up4(up,conv1)
        up = self.conv_last(up)
        up = F.softmax(up,dim=1)
        return up 


class Unet(nn.Module):
    def __init__(self):
        super().__init__()
        self.Down = Down()
        self.Up = Up()

    def forward(self,x):
        conv1,conv2,conv3,conv4,conv5 = self.Down(x)
        x = self.Up(conv1,conv2,conv3,conv4,conv5)
        return x 

In [4]:
class CFG:
    batch_size = 2
    epoch = 50

images,labels = image_data_load() 
train_images,test_images,train_labels,test_labels = train_valid_split(images,labels)

image_transformer = image_transform()
train_dataset = Dset(train_images, train_labels,image_transformer)
test_dataset = Dset(test_images,test_labels,image_transformer)

train_dataloader = DataLoader(train_dataset,CFG.batch_size,shuffle=True)
test_dataloader = DataLoader(test_dataset,CFG.batch_size,shuffle=False)

unet = Unet()
a,b = next(iter(train_dataloader))

2022-08-11 06:35:10.188095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 06:35:10.196122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 06:35:10.196658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 06:35:10.197937: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
unet.to('cuda')
# a = a.to('cuda').type(torch.float)
# a.shape

In [7]:
!nvidia-smi

Thu Aug 11 06:35:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 31%   40C    P2    63W / 250W |  10765MiB / 11018MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------